# Plot NLMs DEMs and Landast images

This notebook produces a figure allowing the comparison of generated NLMs, DEMs and Landsat images for each study site. It assumes that `hydrocorrect_dem.tif` and `init_lct_maps.zip` files (as generated by `make dem` and `make nlm` commands at the root of this repo) exist in the `./outputs/<study-site>` directories.

In [ ]:
from pathlib import Path
import io
from typing import NamedTuple
import zipfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable

from osgeo import gdal
import rasterio
from rasterio.plot import show

from aslib import AgroSuccessLct, Color

from landsat import process_landsat_zip, process_landsat_coll2_tiff

In [ ]:
AS_DATA_ROOT = Path.home() / 'AgroSuccess/agrosuccess-data'
if not AS_DATA_ROOT.is_dir():
    raise FileNotFoundError(f"'{AS_DATA_ROOT}' is not a directory")

In [ ]:
PLOT_DIR = AS_DATA_ROOT / 'landcover-nlms/plots'
if not PLOT_DIR.is_dir():
    PLOT_DIR.mkdir()

## Load a satellite image to guide our design
I'm imagining a plot which has satellite DEM, satelite image and proposed NLM next to each other for illustrative purposes. Satellite image will be needed to get an impression of land cover at that elevation in the present day.

Study site location information is included in the following output

In [ ]:
pd.read_csv(AS_DATA_ROOT / 'outputs/site_location_info.csv')

1. Go to USGS Earth Explorer 
2. In 'Search criteria' enter Polygon -> Add coordinate and specify coordinates of site
3. On 'Data Sets' tab select Landsat -> Landsat Collection 2 Level-1
4. On results tab, select a scene with little cloud cover
5. Download Full-Resolution Browse (Natural Color) GeoTIFF 

For documentation of FRB images see https://www.usgs.gov/landsat-missions/landsat-full-resolution-browse-images and pdf in docs directory. What is called FRB in Landsat Collection 2 was called Landsatlook in Collection 1. 

Record of images downloaded and associated dates:

- Algendar, LC08_L1TP_196032_20230421_20230429_02_T1_refl.tif , 2023/04/21
- Atxuri, LC09_L1TP_200030_20220828_20230331_02_T1_refl.tif, 2022/08/28
- Navarrés, LC08_L1TP_199033_20230426_20230501_02_T1_refl.tif, 2023/04/26
- San Rafael, LC08_L1TP_199034_20230426_20230501_02_T1_refl.tif, 2023/04/26, LC09_L1TP_200034_20230425_20230425_02_T1_refl.tif, 2023/04/25
- Charco da Candieira, LC08_L1TP_203032_20230508_20230517_02_T1_refl.tif, 2023/05/08
- Monte Areo mire, LC08_L1TP_203030_20200718_20200911_02_T1_refl.tif, 2020/07/18

In [ ]:
lsat_file_map = {
    'algendar': 'LC08_L1TP_196032_20230421_20230429_02_T1_refl.tif',
    'atxuri': 'LC09_L1TP_200030_20220828_20230331_02_T1_refl.tif',
    'navarres': 'LC08_L1TP_199033_20230426_20230501_02_T1_refl.tif',
    'san_rafael': 'LC09_L1TP_200034_20230425_20230425_02_T1_refl.tif',
    'charco_da_candieira': 'LC08_L1TP_203032_20230508_20230517_02_T1_refl.tif',
    'monte_areo_mire': 'LC08_L1TP_203030_20200718_20200911_02_T1_refl.tif',
}

In [ ]:
for site, file in lsat_file_map.items():
    process_landsat_coll2_tiff(
        Path(f'data/landsat/raw/{file}'),
        AS_DATA_ROOT / f'outputs/{site}/hydrocorrect_dem.tif',
        Path(f'data/landsat/processed/{site}.tif')
    )

In [ ]:
class SiteData:
    def __init__(self, sitename: str, printname: str, nlm_n: int = 0, as_data_root: Path = AS_DATA_ROOT):
        self.sitename = sitename
        self.printname = printname
        self.nlm_n = nlm_n
        self.as_data_root = as_data_root
        self.lsat_dir = self.as_data_root / 'landcover-nlms/data/landsat'
        
    def _get_site_data_dir(self) -> Path:
        d = self.as_data_root / 'outputs' / self.sitename
        if not d.is_dir():
            raise FileNotFoundError(f"'{d}' is not a directory")
        return d
    
    def _get_dem_file(self) -> Path:
        return self._get_site_data_dir() / 'hydrocorrect_dem.tif'
        
    def get_dem(self) -> np.ndarray:
        f = self._get_dem_file()
        if not f.is_file():
            raise FileNotFoundError(f"'{f}' is not a file")
        with rasterio.open(f) as src:
            return src.read()
        
    def get_nlm(self) -> np.ndarray:
        """Numerical codes correspond to those defined in the AgroSuccessLct
        enum.
        """
        zf = self._get_site_data_dir() / 'init_lct_maps.zip'
        with zipfile.ZipFile(zf, 'r') as z:
            data = io.BytesIO(z.read(f'init-landcover{self.nlm_n}.tif'))
        with rasterio.open(data) as src:
            return src.read()
        
    def get_lsat(self) -> np.ndarray:
        f = self.lsat_dir / f'processed/{self.sitename}.tif'
        if not f.is_file():
            raise FileNotFoundError(f"'{f}' is not a file")
        with rasterio.open(f) as src:
            return src.read()

In [ ]:
ssite_data = [
    SiteData('charco_da_candieira', 'Charco da Candieira'),
    SiteData('atxuri', 'Atxuri'),
    SiteData('monte_areo_mire', 'Monte Areo mire'),
    SiteData('navarres', 'Navarrés'),
    SiteData('algendar', 'Algendar'),
    SiteData('san_rafael', 'San Rafael'),
]

TODO: Add legend for NLMs

In [ ]:
c_list = [x.color.value for x in AgroSuccessLct]
nlm_cmap = ListedColormap(c_list)
# Exclude from legend listing
excluded_lcts = [
    AgroSuccessLct.WATER_QUARRY,
    AgroSuccessLct.WHEAT,
    AgroSuccessLct.DAL,
    AgroSuccessLct.BURNT,
]
include_ixs = [i for i, a in enumerate(AgroSuccessLct) if a not in excluded_lcts]
patches = [
    mpatches.Patch(color=c, label=l)
    for c, l in
    [x for x in zip(
        [c_list[i] for i in include_ixs], [
            x.alias.replace('TransForest', 'Transition Forest')
            for x in AgroSuccessLct if x not in excluded_lcts
        ]
    )]
]

In [ ]:
def make_plots(ssite_data: list[SiteData], figname: str) -> None:
    fig, axes = plt.subplots(
        nrows=len(ssite_data),
        ncols=3,
        figsize=(12, 3.5 * len(ssite_data))
    )
    
    plt.legend(handles=patches, ncols=2, bbox_to_anchor=(-0.01, 3.6))
    for i, site_data in enumerate(ssite_data):
        lsat_ax, nlm_ax, dem_ax = axes[i, :]
        plot_dem_im(dem_ax, site_data.get_dem()[0, :, :])
        plot_lsat_im(lsat_ax, site_data.get_lsat(), site_data.printname)
        plot_nlm_im(nlm_ax, site_data.get_nlm()[0, :, :])
    plt.subplots_adjust(wspace=-0.15, hspace=0.1)
    plt.savefig(PLOT_DIR / figname, dpi=300, bbox_inches='tight')
    

def plot_dem_im(ax, dem_arr):
    im = ax.matshow(dem_arr)
    ax.set_axis_off()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    cbar = fig.colorbar(im, ax=ax, cax=cax)
    cbar.ax.set_ylabel('Elevation [m]', rotation=270, fontsize=12)
    cbar.ax.get_yaxis().labelpad = 16


def plot_lsat_im(ax, lsat_arr, name):
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel(name, fontsize=16)
    show(lsat_arr, ax=ax)
    

def plot_nlm_im(ax, nlm_arr):
    ax.set_axis_off()
    ax.imshow(nlm_arr, cmap=nlm_cmap, vmin=0, vmax=len(nlm_cmap.colors))

In [ ]:
make_plots(ssite_data[:3], 'lsat-nlm-dem-1.png')

In [ ]:
make_plots(ssite_data[3:], 'lsat-nlm-dem-2.png')